In [8]:
import pandas as pd
import numpy as np
import os
import openai
import lmstudio as lms

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score
)
import os

# === Percorsi input/output ===
datasets = {
    "Computer_test": "Test_GPT/result/Computer_test_features_con_llm_predictions.csv",
    "Human_test": "Test_GPT/result/Human_test_features_con_llm_predictions.csv"
}

output_base = "Test_GPT/result"

# === Funzione per valutare e salvare risultati ===
def evaluate_predictions(name, df):
    y_true = df["Label"]
    y_pred = df["LLM_Label"]

    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, zero_division=0)
    rec = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    report = classification_report(y_true, y_pred, zero_division=0)

    # Crea directory se non esiste
    output_dir = os.path.join(output_base, name)
    os.makedirs(output_dir, exist_ok=True)

    # Salva report testuale
    report_path = os.path.join(output_dir, f"evaluation_report_{name}.txt")
    with open(report_path, "w", encoding="utf-8") as f:
        f.write(f"=== Evaluation Report for {name} ===\n")
        f.write(f"Accuracy: {acc:.4f}\n")
        f.write(f"Precision: {prec:.4f}\n")
        f.write(f"Recall: {rec:.4f}\n")
        f.write(f"F1-score: {f1:.4f}\n\n")
        f.write(report)

    # Salva confusion matrix in PDF
    cm = confusion_matrix(y_true, y_pred)
    labels = ["Real (0)", "Fake (1)"]
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', xticklabels=labels, yticklabels=labels, cmap="Blues")
    plt.title(f"Confusion Matrix - {name}")
    plt.xlabel("Predicted")
    plt.ylabel("True")
    cm_path = os.path.join(output_dir, f"confusion_matrix_{name}.pdf")
    plt.tight_layout()
    plt.savefig(cm_path)
    plt.close()

    print(f"✅ Valutazione completata per {name}. Report e matrice salvati in:\n→ {output_dir}\n")

# === Ciclo sui dataset ===
for name, path in datasets.items():
    df = pd.read_csv(path, sep=";", encoding="ISO-8859-1")
    if "LLM_Label" in df.columns:
        evaluate_predictions(name, df)
    else:
        print(f"⚠️ Il file {path} non contiene la colonna 'LLM_Label'")


✅ Valutazione completata per Computer_test. Report e matrice salvati in:
→ Test_GPT/result\Computer_test

✅ Valutazione completata per Human_test. Report e matrice salvati in:
→ Test_GPT/result\Human_test

